In [1]:
import dv_processing as dv
import sys
import cv2
sys.path.append('../')

import numpy as np
import matplotlib.pyplot as plt
from src.data.events_encoding import accumulate_dv_encode
from src.data.preprocessing import *
from src.models.convlstm import ConvLSTM
from src.models.convsnn import ConvSNN
import time
import cv2

# Open any camera
reader = dv.io.MonoCameraRecording("../data/raw_data/raw5.aedat4")



In [3]:
reader.setPlaybackSpeed(1.0)

AttributeError: 'dv_processing.io.MonoCameraRecording' object has no attribute 'setPlaybackSpeed'

In [18]:
import time
import cv2

# Open any camera
reader = dv.io.MonoCameraRecording("../data/raw_data/raw5.aedat4")

# Initialize the previous event timestamp
previous_timestamp = None

# Run the loop while camera is still connected
while reader.isRunning():
    events = reader.getNextEventBatch()
    if events is not None:
        current_timestamp = events[0].timestamp()

        if previous_timestamp is not None:
            time_diff = (current_timestamp - previous_timestamp) / 1e6  # Convert microseconds to seconds

            time.sleep(time_diff)
        print({events.size()})
        previous_timestamp = current_timestamp
        frames = accumulate_dv_encode(events, reader.getEventResolution())

        cv2.imshow("Frames", frames)
    


{2}
{2}
{1}
{1}
{3}
{3}
{6}
{1}
{3}
{1}
{1}
{1}
{3}
{1}
{2}
{4}
{1}
{1}
{1}
{4}
{1}
{1}
{1}
{3}
{1}
{1}
{1}
{2}
{1}
{1}
{1}
{1}
{1}
{1}
{2}
{1}
{1}
{3}
{1}
{1}
{1}
{1}
{1}
{1}
{1}
{2}
{1}
{1}
{1}
{2}
{2}
{1}
{5}
{8}
{3}
{2}
{3}
{6}
{1}
{1}
{1}
{2}
{2}
{1}
{1}
{2}
{1}
{1}
{2}
{6}
{1}
{1}
{2}
{1}
{1}
{1}
{1}
{2}
{1}
{3}
{1}
{2}
{2}
{1}
{4}
{3}
{2}
{1}
{2}
{2}
{2}
{1}
{1}
{2}
{2}
{3}
{1}
{1}
{2}
{1}
{3}
{2}
{1}
{1}
{1}
{4}
{1}
{3}
{2}
{1}
{1}
{1}
{2}
{1}
{1}
{1}
{1}
{2}
{1}
{4}
{5}
{6}


KeyboardInterrupt: 

In [17]:
from datetime import timedelta

store1 = dv.data.generate.uniformEventsWithinTimeRange(10000, timedelta(milliseconds=10), (100, 100), 10)
store1.size()

10

In [5]:
checkpoint_path = "/home/khoahd/Visual-Tactile-Sensing-with-Event-Cameras-using-Spiking-Neural-Networks/saved_models/epoch=8-step=126.ckpt"
model = ConvSNN.load_from_checkpoint(checkpoint_path, beta_init=0.9, in_channels=1, spikegrad="fast_sigmoid", lr=0.01)


In [ ]:
checkpoint_path = "/home/khoahd/Visual-Tactile-Sensing-with-Event-Cameras-using-Spiking-Neural-Networks/saved_models/epoch=8-step=126.ckpt"
model = ConvSNN.load_from_checkpoint(checkpoint_path, beta_init=0.9, in_channels=1, spikegrad="fast_sigmoid", lr=0.01)

list_of_frames = []
for i in range(300):
    frame = torch.zeros((1, 1, 32, 32))
    list_of_frames.append(frame)

# Init mem
mem1 = model.lif1.init_leaky()
mem2 = model.lif2.init_leaky()
mem3 = model.lif3.init_leaky()
mem4 = model.lif4.init_leaky()

for frame in list_of_frames:
    frame = frame.to(model.device)
    
    output, mem1, mem2, mem3, mem4 = model.process_frame(frame, mem1, mem2, mem3, mem4)
    pred = output.argmax(dim=1)
    print(pred)


tensor([1], device='cuda:0')
tensor([1], device='cuda:0')
tensor([1], device='cuda:0')
tensor([1], device='cuda:0')
tensor([1], device='cuda:0')
tensor([1], device='cuda:0')
tensor([1], device='cuda:0')
tensor([1], device='cuda:0')
tensor([1], device='cuda:0')
tensor([1], device='cuda:0')
tensor([1], device='cuda:0')
tensor([1], device='cuda:0')
tensor([1], device='cuda:0')
tensor([1], device='cuda:0')
tensor([1], device='cuda:0')
tensor([1], device='cuda:0')
tensor([1], device='cuda:0')
tensor([1], device='cuda:0')
tensor([1], device='cuda:0')
tensor([1], device='cuda:0')
tensor([1], device='cuda:0')
tensor([1], device='cuda:0')
tensor([1], device='cuda:0')
tensor([1], device='cuda:0')
tensor([1], device='cuda:0')
tensor([1], device='cuda:0')
tensor([1], device='cuda:0')
tensor([1], device='cuda:0')
tensor([1], device='cuda:0')
tensor([1], device='cuda:0')
tensor([1], device='cuda:0')
tensor([1], device='cuda:0')
tensor([1], device='cuda:0')
tensor([1], device='cuda:0')
tensor([1], de